# Benchmarks - Kernels

In MagmaClust and GPs in general, kernels are what makes it possible to compute a covariance matrix for any set of points, given a set of hyperparameters. They are used in many innermost loops, and their implementation is critical for performance.

**Main considerations when implementing kernels**

A good kernel implementation must be:
* fast, as it is used in many innermost loops
* usable at many dimensions, including a batch dimension with distinct hyperparameters for each element
* work on padded inputs (aka inputs with NaNs), maybe using a mask
* jittable, as it is used in many jit-compiled functions
* modular, as kernels can be combined in many ways
* both static and instance-based, to either carry around hyperparameters or be called with them
* easy to override, as users may want to define their own kernels

These goals are conflicting in some cases (e.g: a jittable version of a kernel is not trivial to write for most people), and the best implementation will depend on the specific use case.

---
## Setup

In [1]:
# Standard library
import os
from functools import partial

os.environ['JAX_ENABLE_X64'] = "True"

In [2]:
# Third party
import jax
from jax import numpy as jnp

import numpy as np
import pandas as pd

In [3]:
# Local
from MagmaClustPy.utils import generate_dummy_db, preprocess_db

In [4]:
# Config
key = jax.random.PRNGKey(0)

---
## Data

---
## Current implementation

In [5]:
# import Kernax

---
## Custom implementation(s)

### Defaults of the previous implementation that we wish to correct/improve:

**We want to be able to compose kernels**

As kernels are pytrees, they need to support tree_flatten and tree_unflatten methods.
This is not trivial to implement, as e.g. a sum kernel will have to dynamically know how many hyperparameters belong to each sub-kernel.

In [6]:
import jax.numpy as jnp
from jax import jit, vmap
from jax.tree_util import register_pytree_node_class, tree_map
from jax.tree import reduce
from jax.lax import cond


class StaticAbstractKernel:
	@classmethod
	@partial(jit, static_argnums=(0,))
	def pairwise_cov(cls, kern, x1: jnp.ndarray, x2: jnp.ndarray) -> jnp.ndarray:
		"""
		Compute the kernel covariance value between two vectors.

		:param kern: the kernel to use, containing hyperparameters
		:param x1: scalar array
		:param x2: scalar array
		:param kwargs: hyperparameters of the kernel
		:return: scalar array
		"""
		return jnp.array(jnp.nan)  # To be overwritten in subclasses

	@classmethod
	@partial(jit, static_argnums=(0,))
	def pairwise_cov_if_not_nan(cls, kern, x1: jnp.ndarray, x2: jnp.ndarray) -> jnp.ndarray:
		"""
		Returns NaN if either x1 or x2 is NaN, otherwise calls the compute_scalar method.

		:param kern: the kernel to use, containing hyperparameters
		:param x1: scalar array
		:param x2: scalar array
		:param kwargs: hyperparameters of the kernel
		:return: scalar array
		"""
		return cond(jnp.any(jnp.isnan(x1) | jnp.isnan(x2)),
		            lambda _: jnp.nan,
		            lambda _: cls.pairwise_cov(kern, x1, x2),
		            None)

	@classmethod
	@partial(jit, static_argnums=(0,))
	def cross_cov_vector(cls, kern, x1: jnp.ndarray, x2: jnp.ndarray) -> jnp.ndarray:
		"""
		Compute the kernel cross covariance values between an array of vectors (matrix) and a vector.

		:param kern: the kernel to use, containing hyperparameters
		:param x1: vector array (N, )
		:param x2: scalar array
		:param kwargs: hyperparameters of the kernel
		:return: vector array (N, )
		"""
		return vmap(lambda x: cls.pairwise_cov_if_not_nan(kern, x, x2), in_axes=0)(x1)

	@classmethod
	@partial(jit, static_argnums=(0,))
	def cross_cov_vector_if_not_nan(cls, kern, x1: jnp.ndarray, x2: jnp.ndarray, **kwargs) -> jnp.ndarray:
		"""
		Returns an array of NaN if scalar is NaN, otherwise calls the compute_vector method.

		:param kern: the kernel to use, containing hyperparameters
		:param x1: vector array (N, )
		:param x2: scalar array
		:param kwargs: hyperparameters of the kernel
		:return: vector array (N, )
		"""
		return cond(jnp.any(jnp.isnan(x2)),
		            lambda _: jnp.full(len(x1), jnp.nan),
		            lambda _: cls.cross_cov_vector(kern, x1, x2),
		            None)

	@classmethod
	@partial(jit, static_argnums=(0,))
	def cross_cov_matrix(cls, kern, x1: jnp.ndarray, x2: jnp.ndarray) -> jnp.ndarray:
		"""
		Compute the kernel covariance matrix between two vector arrays.

		:param x1: vector array (N, )
		:param x2: vector array (M, )
		:param kwargs: hyperparameters of the kernel
		:return: matrix array (N, M)
		"""
		return vmap(lambda x: cls.cross_cov_vector_if_not_nan(kern, x2, x), in_axes=0)(x1)

	@classmethod
	@partial(jit, static_argnums=(0,))
	def cross_cov_batch(cls, kern, x1: jnp.ndarray, x2: jnp.ndarray) -> jnp.ndarray:
		"""
		Compute the kernel covariance matrix between two batched vector arrays.

		:param x1: vector array (B, N)
		:param x2: vector array (B, M)
		:param kwargs: hyperparameters of the kernel. Each HP that has the same first dimension as the batch will be considered distinct and thus vmapped over. Other HPs will be shared to the whole batch.
		:return: tensor array (B, N, M)
		"""
		has_input_dim = lambda param: hasattr(param, 'shape') and len(param.shape) > 0 and param.shape[0] == x1.shape[0]
		hp_vmap = tree_map(lambda param: 0 if has_input_dim(param) else None, kern)


		return vmap(lambda k, x, y: cls.cross_cov_matrix(k, x, y), in_axes=(hp_vmap, 0, 0))(kern, x1, x2)


In [7]:
@register_pytree_node_class
class AbstractKernel:
	def __init__(self, **kwargs):
		"""
		Instatiates a kernel with the given hyperparameters.
		https://docs.jax.dev/en/latest/pytrees.html#custom-pytrees-and-initialization
		:param kwargs: the hyperparameters of the kernel, as keyword arguments.
		"""

		# Register hyperparameters in *kwargs* as instance attributes
		self.__dict__.update(kwargs)

		# This check allows the user to assign a static class before and after calling super().__init__()
		if not hasattr(self, 'static_class'):
			self.static_class = StaticAbstractKernel

		if not hasattr(self, 'static_attributes'):
			self.static_attributes = {"static_class", "static_attributes"}
		else:
			self.static_attributes.union({"static_class", "static_attributes"})


	def __str__(self):
		return f"{self.__class__.__name__}({', '.join([f'{key}={value}' for key, value in self.__dict__.items() if key not in self.static_attributes])})"

	def __repr__(self):
		return str(self)

	@jit
	def __call__(self, x1, x2=None):
		# If no x2 is provided, we compute the covariance between x1 and itself
		if x2 is None:
			x2 = x1

		# Turn scalar inputs into vectors
		x1, x2 = jnp.atleast_1d(x1), jnp.atleast_1d(x2)

		# Check for distinct hyperparameters
		kernel_has_some_distinct_hps = reduce(
			lambda acc, param: acc or (hasattr(param, 'shape') and len(param.shape) > 0 and param.shape[0] == x1.shape[0]), self, False)

		if kernel_has_some_distinct_hps and (x1.ndim != 3 or x2.ndim != 3):
			raise ValueError("Kernel with distinct hyperparameters was called on unbatched inputs. It cannot know which hyperparameter value to use for this element")

		# Call the appropriate method
		if jnp.ndim(x1) == 1 and jnp.ndim(x2) == 1:
			return self.static_class.pairwise_cov_if_not_nan(self, x1, x2)
		elif jnp.ndim(x1) == 2 and jnp.ndim(x2) == 1:
			return self.static_class.cross_cov_vector_if_not_nan(self, x1, x2)
		elif jnp.ndim(x1) == 1 and jnp.ndim(x2) == 2:
			return self.static_class.cross_cov_vector_if_not_nan(self, x2, x1)
		elif jnp.ndim(x1) == 2 and jnp.ndim(x2) == 2:
			return self.static_class.cross_cov_matrix(self, x1, x2)
		elif jnp.ndim(x1) == 3 and jnp.ndim(x2) == 3:
			if x1.shape[0] != x2.shape[0]:
				raise ValueError(f"Batch dimension mismatch: x1 has shape {x1.shape}, x2 has shape {x2.shape}.")
			return self.static_class.cross_cov_batch(self, x1, x2)
		else:
			raise ValueError(
				f"Invalid input dimensions: x1 has shape {x1.shape}, x2 has shape {x2.shape}. "
				"Expected 1D, 2D arrays or 3D arrays for batched inputs."
			)

	# Methods to use Kernel as a PyTree
	def tree_flatten(self):
		return tuple(val for key, val in self.__dict__.items() if key not in self.static_attributes), None  # No static values

	@classmethod
	def tree_unflatten(cls, _, children):
		# This class being abstract, this function fails when called on an "abstract instance",
		# as we don't know the number of parameters the constructor expects, yet we send it children.
		# On a subclass, this will work as expected as long as the constructor has a clear number of
		# kwargs as parameters.
		return cls(*children)

	def __add__(self, other):
		#from Kernax.OperatorKernels import SumKernel
		return SumKernel(self, other)

	def __radd__(self, other):
		#from Kernax.OperatorKernels import SumKernel
		return SumKernel(other, self)

	def __neg__(self):
		#from Kernax.WrapperKernels import NegKernel
		return NegKernel(self)

	def __mul__(self, other):
		#from Kernax.OperatorKernels import ProductKernel
		return ProductKernel(self, other)

	def __rmul__(self, other):
		#from Kernax.OperatorKernels import ProductKernel
		return ProductKernel(other, self)


In [8]:
class StaticConstantKernel(StaticAbstractKernel):
	@classmethod
	@partial(jit, static_argnums=(0,))
	def pairwise_cov(cls, kern, x1: jnp.ndarray, x2: jnp.ndarray) -> jnp.ndarray:
		"""
		Compute the kernel covariance value between two vectors.

		:param kern: the kernel to use, containing hyperparameters
		:param x1: scalar array
		:param x2: scalar array
		:param kwargs: hyperparameters of the kernel
		:return: scalar array
		"""
		return kern.value  # The constant value is returned regardless of the inputs


@register_pytree_node_class
class ConstantKernel(AbstractKernel):
	def __init__(self, value=1.):
		"""
		Instantiates a constant kernel with the given value.

		:param value: the value of the constant kernel
		"""
		super().__init__(value=value)
		self.static_class = StaticConstantKernel


In [9]:
class StaticSEMagmaKernel(StaticAbstractKernel):
	@classmethod
	@partial(jit, static_argnums=(0,))
	def pairwise_cov(cls, kern, x1: jnp.ndarray, x2: jnp.ndarray) -> jnp.ndarray:
		"""
		Compute the kernel covariance value between two vectors.

		:param kern: the kernel to use, containing hyperparameters
		:param x1: scalar array
		:param x2: scalar array
		:param kwargs: hyperparameters of the kernel
		:return: scalar array
		"""
		return jnp.exp(kern.variance - jnp.exp(-kern.length_scale) * jnp.sum((x1 - x2) ** 2) * 0.5)

@register_pytree_node_class
class SEMagmaKernel(AbstractKernel):
	def __init__(self, length_scale=None, variance=None, **kwargs):
		super().__init__(length_scale=length_scale, variance=variance, **kwargs)
		self.static_class = StaticSEMagmaKernel

In [10]:
import jax.numpy as jnp
from jax import jit
from jax.tree_util import register_pytree_node_class

# from Kernax import AbstractKernel, ConstantKernel


@register_pytree_node_class
class OperatorKernel(AbstractKernel):
	""" Class for kernels that apply some operation on the output of two kernels."""
	def __init__(self, left_kernel, right_kernel):
		"""
		Instantiates a sum kernel with the given kernels.

		:param right_kernel: the right kernel to sum
		:param left_kernel: the left kernel to sum
		"""
		# If any of the provided arguments are not kernels, we try to convert them to ConstantKernels
		if not isinstance(left_kernel, AbstractKernel):
			left_kernel = ConstantKernel(value=left_kernel)
		if not isinstance(right_kernel, AbstractKernel):
			right_kernel = ConstantKernel(value=right_kernel)

		super().__init__(left_kernel=left_kernel, right_kernel=right_kernel)

@register_pytree_node_class
class SumKernel(OperatorKernel):
	""" Sum kernel that sums the outputs of two kernels."""
	@jit
	def __call__(self, x1: jnp.ndarray, x2: jnp.ndarray = None) -> jnp.ndarray:
		if x2 is None:
			x2 = x1

		return self.left_kernel(x1, x2) + self.right_kernel(x1, x2)


@register_pytree_node_class
class ProductKernel(OperatorKernel):
	""" Product kernel that multiplies the outputs of two kernels. """

	@jit
	def __call__(self, x1: jnp.ndarray, x2: jnp.ndarray = None) -> jnp.ndarray:
		if x2 is None:
			x2 = x1

		return self.left_kernel(x1, x2) * self.right_kernel(x1, x2)


In [11]:
import jax.numpy as jnp
from jax import jit
from jax.tree_util import register_pytree_node_class
from jax.lax import cond

#from Kernax import AbstractKernel, ConstantKernel


@register_pytree_node_class
class WrapperKernel(AbstractKernel):
	""" Class for kernels that perform some operation on the output of another "inner" kernel."""
	def __init__(self, inner_kernel=None):
		"""
		Instantiates a wrapper kernel with the given inner kernel.

		:param inner_kernel: the inner kernel to wrap
		:param kwargs: hyperparameters of the kernel
		"""
		# If the inner kernel is not a kernel, we try to convert it to a ConstantKernel
		if not isinstance(inner_kernel, AbstractKernel):
			inner_kernel = ConstantKernel(value=inner_kernel)

		super().__init__(inner_kernel=inner_kernel)

class StaticDiagKernel(StaticAbstractKernel):
	"""
	Static kernel that returns a value only if the inputs are equal, otherwise returns 0.
	This results in a diagonal cross-covariance matrix.
	"""
	@classmethod
	@partial(jit, static_argnums=(0,))
	def pairwise_cov(cls, kern, x1: jnp.ndarray, x2: jnp.ndarray) -> jnp.ndarray:
		return cond(jnp.all(x1 == x2),
		            lambda _: kern.inner_kernel(x1, x2),
		            lambda _: jnp.array(0.0),
		            None)


@register_pytree_node_class
class DiagKernel(WrapperKernel):
	"""
	Kernel that returns a value only if the inputs are equal, otherwise returns 0.
	This results in a diagonal cross-covariance matrix.
	"""
	def __init__(self, inner_kernel=None):
		super().__init__(inner_kernel=inner_kernel)
		self.static_class = StaticDiagKernel


@register_pytree_node_class
class ExpKernel(WrapperKernel):
	"""
	Kernel that applies the exponential operator to the output of another kernel.
	"""
	@jit
	def __call__(self, x1: jnp.ndarray, x2: jnp.ndarray = None) -> jnp.ndarray:
		if x2 is None:
			x2 = x1

		return jnp.exp(self.inner_kernel(x1, x2))


@register_pytree_node_class
class LogKernel(WrapperKernel):
	"""
	Kernel that applies the logarithm operator to the output of another kernel.
	"""
	@jit
	def __call__(self, x1: jnp.ndarray, x2: jnp.ndarray = None) -> jnp.ndarray:
		if x2 is None:
			x2 = x1

		return jnp.log(self.inner_kernel(x1, x2))


@register_pytree_node_class
class NegKernel(WrapperKernel):
	@jit
	def __call__(self, x1: jnp.ndarray, x2: jnp.ndarray = None) -> jnp.ndarray:
		if x2 is None:
			x2 = x1

		return - self.inner_kernel(x1, x2)


---
## Comparison

### On padded datasets

In [12]:
db = pd.read_csv("../datasets/small_distinct_input_shared_hp.csv")
padded_inputs, padded_outputs, mappings, all_inputs = preprocess_db(db)
padded_inputs.shape, padded_outputs.shape, mappings.shape, all_inputs.shape

((20, 15, 1), (20, 15, 1), (20, 15), (41, 1))

In [13]:
noises = jnp.arange(1., 2., .05)
noises.shape

(20,)

In [14]:
ExpKernel(jnp.arange(1., 21, 1))(padded_inputs)

Array([[[2.71828183e+00, 2.71828183e+00, 2.71828183e+00, ...,
         2.71828183e+00, 2.71828183e+00, 2.71828183e+00],
        [2.71828183e+00, 2.71828183e+00, 2.71828183e+00, ...,
         2.71828183e+00, 2.71828183e+00, 2.71828183e+00],
        [2.71828183e+00, 2.71828183e+00, 2.71828183e+00, ...,
         2.71828183e+00, 2.71828183e+00, 2.71828183e+00],
        ...,
        [2.71828183e+00, 2.71828183e+00, 2.71828183e+00, ...,
         2.71828183e+00, 2.71828183e+00, 2.71828183e+00],
        [2.71828183e+00, 2.71828183e+00, 2.71828183e+00, ...,
         2.71828183e+00, 2.71828183e+00, 2.71828183e+00],
        [2.71828183e+00, 2.71828183e+00, 2.71828183e+00, ...,
         2.71828183e+00, 2.71828183e+00, 2.71828183e+00]],

       [[7.38905610e+00, 7.38905610e+00, 7.38905610e+00, ...,
         7.38905610e+00, 7.38905610e+00, 7.38905610e+00],
        [7.38905610e+00, 7.38905610e+00, 7.38905610e+00, ...,
         7.38905610e+00, 7.38905610e+00, 7.38905610e+00],
        [7.38905610e+00, 

In [15]:
#old_kernel = Kernax.SEMagmaKernel(length_scale=jnp.array(.3), variance=jnp.array(1.))
new_kernel = SEMagmaKernel(length_scale=.3, variance=1.) + DiagKernel(ExpKernel(noises))

In [16]:
k = ExpKernel(1.)
k.inner_kernel

ConstantKernel(value=1.0)

In [17]:
k.inner_kernel(jnp.array([1.]), jnp.array([2.]))

Array(1., dtype=float64, weak_type=True)

In [18]:
k.inner_kernel.static_class.pairwise_cov(k.inner_kernel, jnp.array([1.]), jnp.array([2.]))

Array(1., dtype=float64, weak_type=True)

In [19]:
k.inner_kernel.value

1.0

In [25]:
np.asarray(new_kernel(padded_inputs)[0])

array([[5.43656366e+00, 1.87684259e+00, 1.18125744e+00, 9.69381635e-02,
        3.69895326e-05, 4.99744738e-18, 9.32034373e-20, 1.44438281e-21,
        1.30722600e-29, 8.02480483e-32, 4.09340734e-34, 1.78828412e-41,
        4.34864282e-44, 2.38610445e-55, 1.83974406e-61],
       [1.87684259e+00, 5.43656366e+00, 2.47786604e+00, 6.17771617e-01,
        1.50226023e-03, 8.24291618e-15, 2.22654366e-16, 4.99744738e-18,
        1.99013770e-25, 1.76943071e-27, 1.30722600e-29, 1.73501060e-36,
        6.11062994e-39, 1.47532679e-49, 2.38610445e-55],
       [1.18125744e+00, 2.47786604e+00, 5.43656366e+00, 1.18125744e+00,
        7.25150848e-03, 2.53569629e-13, 8.24291618e-15, 2.22654366e-16,
        1.85994190e-23, 1.99013770e-25, 1.76943071e-27, 4.09340734e-34,
        1.73501060e-36, 8.78693960e-47, 2.05828495e-52],
       [9.69381635e-02, 6.17771617e-01, 1.18125744e+00, 5.43656366e+00,
        2.68459037e-01, 2.42966498e-09, 1.37667045e-10, 6.48157084e-12,
        4.99744738e-18, 9.32034373e-2

In [21]:
np.asarray(new_kernel.right_kernel(padded_inputs)[0])

array([[2.71828183, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 2.71828183, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 2.71828183, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 2.71828183, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 2.71828183,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 

In [22]:
np.asarray(new_kernel.right_kernel(padded_inputs)[0])

array([[2.71828183, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 2.71828183, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 2.71828183, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 2.71828183, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 2.71828183,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 

In [23]:
new_kernel.right_kernel.inner_kernel.inner_kernel

ConstantKernel(value=[1.   1.05 1.1  1.15 1.2  1.25 1.3  1.35 1.4  1.45 1.5  1.55 1.6  1.65
 1.7  1.75 1.8  1.85 1.9  1.95])

In [24]:
0/0

ZeroDivisionError: division by zero

In [14]:
# Covariance on padded matrix
res1 = old_kernel(padded_inputs)
res1.shape

NameError: name 'old_kernel' is not defined

In [20]:
# Covariance on un-padded matrix
# res2 = new_kernel(padded_inputs)
res2 = new_kernel(padded_inputs)
res2.shape

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x120d477d0>>
Traceback (most recent call last):
  File "/opt/anaconda3/envs/MagmaClustPy/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


ValueError: vmap was requested to map its argument along axis 0, which implies that its rank should be at least 1, but is only 0 (its shape is ())

In [26]:
# Check that values in un-padded matrix correspond to the values in the padded matrix
jnp.allclose(jnp.nan_to_num(res1), jnp.nan_to_num(res2))

Array(False, dtype=bool)

In [39]:
%timeit old_kernel(padded_inputs.squeeze()).block_until_ready()

60.1 μs ± 1.07 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [40]:
%timeit new_kernel(padded_inputs).block_until_ready()

46.4 μs ± 1.03 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [41]:
%timeit old_kernel(padded_inputs[0].squeeze()).block_until_ready()

96.3 μs ± 1.4 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [42]:
%timeit new_kernel(padded_inputs[0]).block_until_ready()

86.9 μs ± 1.82 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [20]:
%timeit new_kernel.cross_covariance(padded_inputs[0].reshape(-1, 1), padded_inputs[0].reshape(-1, 1)).block_until_ready()

2.41 ms ± 49.3 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### On scalars

In [26]:
key, subkey = jax.random.split(key)
a = jax.random.uniform(subkey, ())
key, subkey = jax.random.split(key)
b = jax.random.uniform(subkey, ())
a, b

(Array(0.67119299, dtype=float64), Array(0.35166634, dtype=float64))

In [27]:
res1 = old_kernel(a, b)
np.asarray(res1)

array(0.56710711)

In [28]:
res2 = new_kernel(a.reshape(-1, 1), b.reshape(-1, 1))
np.asarray(res2)

array(0.56710711)

In [29]:
jnp.allclose(res1, res2)

Array(True, dtype=bool)

In [30]:
%timeit old_kernel(a, b).block_until_ready()

5.78 μs ± 29.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [31]:
%timeit new_kernel(a.reshape(-1, 1), b.reshape(-1, 1)).block_until_ready()

147 μs ± 1.39 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### On an array and a scalar

In [32]:
key, subkey = jax.random.split(key)
a = jax.random.uniform(subkey, (10000,))
key, subkey = jax.random.split(key)
b = jax.random.uniform(subkey, ())
a, b

(Array([0.15974131, 0.14370377, 0.62062701, ..., 0.7647444 , 0.49158483,
        0.34969892], dtype=float64),
 Array(0.96106866, dtype=float64))

In [33]:
res1 = old_kernel(a, b)
np.asarray(res1)

array([0.02823017, 0.02443875, 0.52524432, ..., 0.80724426, 0.29389634,
       0.12536618])

In [34]:
res2 = new_kernel(a, b)
np.asarray(res2)

IndexError: Too many indices: 0-dimensional array indexed with 1 regular index.

In [42]:
jnp.allclose(res1, res2)

Array(False, dtype=bool)

In [43]:
%timeit old_kernel(a, b).block_until_ready()

47.1 ms ± 3.44 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [44]:
%timeit new_kernel(a, b).block_until_ready()

33.9 ms ± 9.22 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### On two arrays

In [45]:
key, subkey = jax.random.split(key)
a = jax.random.uniform(subkey, (10000,))
key, subkey = jax.random.split(key)
b = jax.random.uniform(subkey, (15000,))
a.shape, b.shape

((10000,), (15000,))

In [46]:
res1 = old_kernel(a, b)
np.asarray(res1).shape

(10000, 15000)

In [47]:
res2 = new_kernel(a, b)
np.asarray(res2).shape

(10000, 15000)

In [48]:
jnp.allclose(res1, res2)

Array(False, dtype=bool)

In [49]:
%timeit old_kernel(a, b).block_until_ready()

1min 10s ± 5.42 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [50]:
%timeit new_kernel(a, b).block_until_ready()

14.5 s ± 4.42 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


### On two batches of arrays with shared HP

In [51]:
key, subkey = jax.random.split(key)
a = jax.random.uniform(subkey, (50, 100))
key, subkey = jax.random.split(key)
b = jax.random.uniform(subkey, (50, 150))
a.shape, b.shape

((50, 100), (50, 150))

In [52]:
# Using shared hyperparameters for all batches
res1 = old_kernel(a, b)
np.asarray(res1).shape

{'inner_kernel': NewSEMagmaKernel(length_scale=0.3, variance=1.0), 'noise': Array(-2.5, dtype=float64, weak_type=True)} {'length_scale': Array(0.3, dtype=float64, weak_type=True), 'variance': Array(1., dtype=float64, weak_type=True)} {'inner_kernel': NewSEMagmaKernel(length_scale=0.3, variance=1.0), 'noise': Array(-2.5, dtype=float64, weak_type=True), 'length_scale': Array(0.3, dtype=float64, weak_type=True), 'variance': Array(1., dtype=float64, weak_type=True)}


(50, 100, 150)

In [53]:
# Also using shared hyperparameters for all batches
res2 = new_kernel(a, b)
np.asarray(res2).shape

{'length_scale': Array(1., dtype=float64, weak_type=True), 'variance': Array(1., dtype=float64, weak_type=True)} {} {'length_scale': Array(1., dtype=float64, weak_type=True), 'variance': Array(1., dtype=float64, weak_type=True)}


(50, 100, 150)

In [54]:
jnp.allclose(res1, res2)

Array(False, dtype=bool)

In [55]:
%timeit old_kernel(a, b).block_until_ready()

{'inner_kernel': NewSEMagmaKernel(length_scale=0.3, variance=1.0), 'noise': Array(-2.5, dtype=float64, weak_type=True)} {'length_scale': Array(0.3, dtype=float64, weak_type=True), 'variance': Array(1., dtype=float64, weak_type=True)} {'inner_kernel': NewSEMagmaKernel(length_scale=0.3, variance=1.0), 'noise': Array(-2.5, dtype=float64, weak_type=True), 'length_scale': Array(0.3, dtype=float64, weak_type=True), 'variance': Array(1., dtype=float64, weak_type=True)}
{'inner_kernel': NewSEMagmaKernel(length_scale=0.3, variance=1.0), 'noise': Array(-2.5, dtype=float64, weak_type=True)} {'length_scale': Array(0.3, dtype=float64, weak_type=True), 'variance': Array(1., dtype=float64, weak_type=True)} {'inner_kernel': NewSEMagmaKernel(length_scale=0.3, variance=1.0), 'noise': Array(-2.5, dtype=float64, weak_type=True), 'length_scale': Array(0.3, dtype=float64, weak_type=True), 'variance': Array(1., dtype=float64, weak_type=True)}
{'inner_kernel': NewSEMagmaKernel(length_scale=0.3, variance=1.0),

In [56]:
%timeit new_kernel(a, b).block_until_ready()

{'length_scale': Array(1., dtype=float64, weak_type=True), 'variance': Array(1., dtype=float64, weak_type=True)} {} {'length_scale': Array(1., dtype=float64, weak_type=True), 'variance': Array(1., dtype=float64, weak_type=True)}
{'length_scale': Array(1., dtype=float64, weak_type=True), 'variance': Array(1., dtype=float64, weak_type=True)} {} {'length_scale': Array(1., dtype=float64, weak_type=True), 'variance': Array(1., dtype=float64, weak_type=True)}
{'length_scale': Array(1., dtype=float64, weak_type=True), 'variance': Array(1., dtype=float64, weak_type=True)} {} {'length_scale': Array(1., dtype=float64, weak_type=True), 'variance': Array(1., dtype=float64, weak_type=True)}
{'length_scale': Array(1., dtype=float64, weak_type=True), 'variance': Array(1., dtype=float64, weak_type=True)} {} {'length_scale': Array(1., dtype=float64, weak_type=True), 'variance': Array(1., dtype=float64, weak_type=True)}
{'length_scale': Array(1., dtype=float64, weak_type=True), 'variance': Array(1., dty

### On two batches of arrays with distinct HP

In [57]:
key, subkey = jax.random.split(key)
a = jax.random.uniform(subkey, (50, 100))  # 5 batches of 10-dimensional data
key, subkey = jax.random.split(key)
b = jax.random.uniform(subkey, (50, 150))  # 5 batches of 15-dimensional data

# Create distinct hyperparameters for each batch
key, subkey = jax.random.split(key)
distinct_length_scales = jax.random.uniform(subkey, (50,)) + 0.5  # Ensuring positive values
key, subkey = jax.random.split(key)
distinct_variances = jax.random.uniform(subkey, (50,)) + 0.5  # Ensuring positive values

distinct_length_scales, distinct_variances

(Array([1.40087618, 0.50347402, 0.81557193, 1.34923058, 1.00457197,
        0.65788761, 0.9219105 , 0.72545942, 1.23310028, 0.91450771,
        0.52153169, 1.07353706, 0.84001156, 0.63915561, 1.28793987,
        0.960267  , 0.5185922 , 1.46299715, 1.2229984 , 0.73096194,
        1.47447511, 1.41269148, 0.62063165, 1.39239143, 0.56357474,
        0.95879739, 1.23934137, 0.66682831, 1.27697895, 1.41619929,
        1.1099631 , 1.14399616, 1.30613247, 0.78241536, 1.13752401,
        0.7997413 , 0.86489189, 1.1392135 , 1.05768611, 0.64485643,
        0.78303156, 0.61316633, 0.53466457, 0.93155447, 0.81706077,
        0.89970457, 0.6447488 , 1.16024019, 1.30514671, 0.9256145 ],      dtype=float64),
 Array([1.34874564, 1.01689555, 1.11981318, 0.96395389, 0.73946461,
        0.57796114, 0.86174911, 0.87934257, 1.08225335, 1.42074674,
        1.38278328, 0.69312996, 0.62528442, 0.52303255, 1.35397314,
        0.62574799, 1.25923923, 0.82557538, 1.32550512, 1.13693679,
        0.61045989, 0.9583

In [58]:
res1 = old_kernel(a, b, length_scale=distinct_length_scales, variance=distinct_variances)

{'length_scale': Array([1.40087618, 0.50347402, 0.81557193, 1.34923058, 1.00457197,
       0.65788761, 0.9219105 , 0.72545942, 1.23310028, 0.91450771,
       0.52153169, 1.07353706, 0.84001156, 0.63915561, 1.28793987,
       0.960267  , 0.5185922 , 1.46299715, 1.2229984 , 0.73096194,
       1.47447511, 1.41269148, 0.62063165, 1.39239143, 0.56357474,
       0.95879739, 1.23934137, 0.66682831, 1.27697895, 1.41619929,
       1.1099631 , 1.14399616, 1.30613247, 0.78241536, 1.13752401,
       0.7997413 , 0.86489189, 1.1392135 , 1.05768611, 0.64485643,
       0.78303156, 0.61316633, 0.53466457, 0.93155447, 0.81706077,
       0.89970457, 0.6447488 , 1.16024019, 1.30514671, 0.9256145 ],      dtype=float64), 'variance': Array([1.34874564, 1.01689555, 1.11981318, 0.96395389, 0.73946461,
       0.57796114, 0.86174911, 0.87934257, 1.08225335, 1.42074674,
       1.38278328, 0.69312996, 0.62528442, 0.52303255, 1.35397314,
       0.62574799, 1.25923923, 0.82557538, 1.32550512, 1.13693679,
       0.61

In [59]:
res2 = new_kernel(a, b, length_scale=distinct_length_scales, variance=distinct_variances)

{'length_scale': Array([1.40087618, 0.50347402, 0.81557193, 1.34923058, 1.00457197,
       0.65788761, 0.9219105 , 0.72545942, 1.23310028, 0.91450771,
       0.52153169, 1.07353706, 0.84001156, 0.63915561, 1.28793987,
       0.960267  , 0.5185922 , 1.46299715, 1.2229984 , 0.73096194,
       1.47447511, 1.41269148, 0.62063165, 1.39239143, 0.56357474,
       0.95879739, 1.23934137, 0.66682831, 1.27697895, 1.41619929,
       1.1099631 , 1.14399616, 1.30613247, 0.78241536, 1.13752401,
       0.7997413 , 0.86489189, 1.1392135 , 1.05768611, 0.64485643,
       0.78303156, 0.61316633, 0.53466457, 0.93155447, 0.81706077,
       0.89970457, 0.6447488 , 1.16024019, 1.30514671, 0.9256145 ],      dtype=float64), 'variance': Array([1.34874564, 1.01689555, 1.11981318, 0.96395389, 0.73946461,
       0.57796114, 0.86174911, 0.87934257, 1.08225335, 1.42074674,
       1.38278328, 0.69312996, 0.62528442, 0.52303255, 1.35397314,
       0.62574799, 1.25923923, 0.82557538, 1.32550512, 1.13693679,
       0.61

In [60]:
jnp.allclose(res1, res2)

Array(False, dtype=bool)

In [61]:
%timeit old_kernel(a, b, length_scale=distinct_length_scales, variance=distinct_variances).block_until_ready()

{'length_scale': Array([1.40087618, 0.50347402, 0.81557193, 1.34923058, 1.00457197,
       0.65788761, 0.9219105 , 0.72545942, 1.23310028, 0.91450771,
       0.52153169, 1.07353706, 0.84001156, 0.63915561, 1.28793987,
       0.960267  , 0.5185922 , 1.46299715, 1.2229984 , 0.73096194,
       1.47447511, 1.41269148, 0.62063165, 1.39239143, 0.56357474,
       0.95879739, 1.23934137, 0.66682831, 1.27697895, 1.41619929,
       1.1099631 , 1.14399616, 1.30613247, 0.78241536, 1.13752401,
       0.7997413 , 0.86489189, 1.1392135 , 1.05768611, 0.64485643,
       0.78303156, 0.61316633, 0.53466457, 0.93155447, 0.81706077,
       0.89970457, 0.6447488 , 1.16024019, 1.30514671, 0.9256145 ],      dtype=float64), 'variance': Array([1.34874564, 1.01689555, 1.11981318, 0.96395389, 0.73946461,
       0.57796114, 0.86174911, 0.87934257, 1.08225335, 1.42074674,
       1.38278328, 0.69312996, 0.62528442, 0.52303255, 1.35397314,
       0.62574799, 1.25923923, 0.82557538, 1.32550512, 1.13693679,
       0.61

In [62]:
%timeit new_kernel(a, b, length_scale=distinct_length_scales, variance=distinct_variances).block_until_ready()

{'length_scale': Array([1.40087618, 0.50347402, 0.81557193, 1.34923058, 1.00457197,
       0.65788761, 0.9219105 , 0.72545942, 1.23310028, 0.91450771,
       0.52153169, 1.07353706, 0.84001156, 0.63915561, 1.28793987,
       0.960267  , 0.5185922 , 1.46299715, 1.2229984 , 0.73096194,
       1.47447511, 1.41269148, 0.62063165, 1.39239143, 0.56357474,
       0.95879739, 1.23934137, 0.66682831, 1.27697895, 1.41619929,
       1.1099631 , 1.14399616, 1.30613247, 0.78241536, 1.13752401,
       0.7997413 , 0.86489189, 1.1392135 , 1.05768611, 0.64485643,
       0.78303156, 0.61316633, 0.53466457, 0.93155447, 0.81706077,
       0.89970457, 0.6447488 , 1.16024019, 1.30514671, 0.9256145 ],      dtype=float64), 'variance': Array([1.34874564, 1.01689555, 1.11981318, 0.96395389, 0.73946461,
       0.57796114, 0.86174911, 0.87934257, 1.08225335, 1.42074674,
       1.38278328, 0.69312996, 0.62528442, 0.52303255, 1.35397314,
       0.62574799, 1.25923923, 0.82557538, 1.32550512, 1.13693679,
       0.61

### On two batches of arrays with both shared and distinct HP

---
## Conclusion

The conditional logic in `compute_vector` that simplifies computations on NaNs gives us roughly a 2x improvement in speed on padded matrices. However, this solution is still 6x slower than computing the covariance batch on a "denser" input. *In fine*, computing a batch kernel is a $O(M*N^2)$ operation. Computing covariance on smallest possible (meaning "padded but not aligned") vectors (N = 90) is obviously faster than on fully padded and aligned ones (N = 400).

It is interesting to note that this speed gain is only obtained if no implementation of `compute_matrix` and `compute_batch` is given, as those implementations would skip `compute_vector`.

**In the future, we should explore whether computing kernels on "compact" padded inputs and then map the values to an "aligned" batch preserves this speed gain. If so, it might be useful to consider using this "compact" representation throughout the whole algorithm.**

---